
## PyAutoGUI&PyWinAuto

## ![](http://winsupersite.com/content/content/128528/win7/win7_ff_calc_rtm_02.jpg)

這篇文章主要說明遇到 Windows native UI 時候或是Windows Application 時，如何進行自動化測試?

Selenium的自動化測試僅限於 Web UI，對於 Windows UI 或是 Windows 應用程式(例如小算盤)往往無法處理。

有什麼方式可以直接在 python 處理Windows UI 而不需要額外呼叫外部 AutoIT 所編譯的 EXE?

這篇文章介紹兩個方式，不儘 Windows 適用，Mac/Linux 也可以。最後給完整的程式範例說明。

## 方法一：畫面圖形比對的方式 PyAutoGUI

工具主角:PyAutoGUI

### 安裝方式

<table><tbody><tr><td>pip &nbsp; &nbsp;install &nbsp; PyAutoGUI</td></tr></tbody></table>

### 優缺點

這個方式最大的好處是跨平台 Windows, Mac, Linux 都可以適用，因為採用圖形比對的關係也比較直覺。

缺點就是圖形比對來找畫面元件的操作方式，誤判或是失敗的機率會比較高。

另外，由於主要是用滑鼠作標的關係，因此螢幕解析度也有可能會影響程式的進行。

### 實作範例

我們主要用小算盤為例子。舉例來說，如果我們要點擊小算盤的 “7”。

我們可以截取一張圖如下，檔名為cal7.png。(檔案格式一定要是PNG)

![_images/calc7key.png](https://pyautogui.readthedocs.org/en/latest/_images/calc7key.png)

執行下列 python 程式後，就會點擊到 7
```
import pyautogui
#pyautogui.moveTo( pyautogui.size(),duration=2)

button7location = pyautogui.locateOnScreen('c:\\cal7.png')

pyautogui.click(button7location)
```

要注意的是如果執行時小算盤被其他畫面遮到，那麼執行就會失敗。

因此，執行這類的程式時，必須要確保小算盤在螢幕上可以被看見。沒有被其他 Window 遮住。

### 技術參考資料

[https://pypi.python.org/pypi/PyAutoGUI](https://pypi.python.org/pypi/PyAutoGUI)

[https://pyautogui.readthedocs.org/en/latest/index.html](https://pyautogui.readthedocs.org/en/latest/index.html)

## 分法二: Windows ObjectID 的方式 PyWinAuto + SWAPY

工具主角: PyWinAuto + SWAPY

### 安裝方式

1\. 首先安裝 PyWinAuto

<table style="height: 32px;" width="250"><tbody><tr><td>pip &nbsp; install &nbsp; &nbsp;pyWinAuto</td></tr></tbody></table>

2\. 下載 SWAPY&nbsp;([Simple Windows Automation on Python](https://code.google.com/p/swapy/))

SWAPY 為一個獨立的執行工具 ，可以在這裡下載安裝：

[https://swapy.googlecode.com/files/swapy-ob-0.4.3.exe](https://swapy.googlecode.com/files/swapy-ob-0.4.3.exe)

### 優缺點

PyWinAuto&nbsp;這個方式主要透過 Windows Object locator 來定位 Windows 元件。

缺點是只能限定在 Windows 平台使用。

優點是透過 ObjectID 的方式定位，比較不會像圖形比對那樣失敗或是受到畫面修改的影響。

另外一個最大的優點是，我們可以透過 “SWAPY”這個工具自動產生程式!

## 測試情境

我們主要會自動執行下列動作：

1\. 啟動 notepad

2\. 用電腦自動點選 Help > About Notepad

## 為什麼用 SWAPY實作

雖然我們可以直接寫 PyWinAuto 的程式，但是筆者還是建議用 SWAPY 這個工具來直接產生。因為這個工具 SWAPY 就好比是瀏覽器的 &nbsp;Web element inspector ，可以看 Windows 物件的狀況，幫助我們定位並且做適當的動作 (click..)

1\. 首先先啟動 notepad.exe

2\. 接著再啟動 SWAPY。

因為 SWAPY 沒有 refresh 的功能，所以如果先啟動 SWAPY 之後再啟動 notepad的話，SWAPY 會找不到 notepad.exe

如果開啟的視窗比較多的話，一開始啟動的時間會比較久一些。

## SWAPY 實作使用步驟

1\. 啟動SWAPY之後，找到 “Untitled – Notepad” > 右鍵 > “Set Focus”

2\. 展開 menu > &About Notepad > &nbsp;右鍵 > “(click” 如下圖所示)

3\. SWAPY 就會自動完成，右手邊 Editor 的相關 python 程式! 是不是很方便。

[![SWAPY use1](images/SWAPY-use1.png)

由於notepad.exe 是我們手動所啟動，所以要將該 python 程式自動化執行，每次都會自動啟動 notepad

所以上述自動產生程式最上面再加入這行：

<table><tbody><tr><td>pwa_app.start_(‘notepad.exe’)</td></tr></tbody></table>

## 完整Python程式碼範例 (SWAPY自動產生)

```
import pywinauto


pwa_app = pywinauto.application.Application()

# Add this line to launch notepad.exe. The other parts of source code were generated by SWAPY
pwa_app.start_('notepad.exe')

w_handle = pywinauto.findwindows.find_windows(title=u'Untitled - Notepad', class_name='Notepad')[0]
window = pwa_app.window_(handle=w_handle)
window.SetFocus()
window.MenuItem(u'&Help->&About Notepad').Click()
```
[http://pywinauto.github.io/docs/](http://pywinauto.github.io/docs/)

[https://github.com/pywinauto/pywinauto](https://github.com/pywinauto/pywinauto)

[https://github.com/pywinauto/pywinauto/tree/f0e892cce6f367d94b675bb1b9e5db20f0425747/examples](https://github.com/pywinauto/pywinauto/tree/f0e892cce6f367d94b675bb1b9e5db20f0425747/examples)

## AutoIT

另外可以使用AutoIT，將自動化程式編譯成EXE

[https://www.autoitscript.com/site/autoit/](https://www.autoitscript.com/site/autoit/)

## PyAutoGUI與PyWinAuto比較

<table style="height: 307px;" width="833"><tbody><tr><td>工具</td><td>**PyAutoGUI**</td><td>**PyWinAuto + SWAPY**</td></tr><tr><td>定位方式</td><td>透過圖形PNG</td><td>&nbsp;Windows objectID</td></tr><tr><td>實作</td><td>必須要透過截圖程式抓取要點選的畫面</td><td>使用SWAPY工具</td></tr><tr><td>程式失敗的可能原因</td><td>圖形比對失敗螢幕解析度該畫面被其他視窗遮蓋</td><td>相對穩定</td></tr><tr><td>適用平台</td><td>Mac, Linux, Windows</td><td>Windows Only</td></tr><tr><td>適用應用程式</td><td>因為是圖形比對的方式，所以網頁、Java Applet、Flash、Windows UI、Mac UI等都可以適用</td><td>Windows Application only</td></tr></tbody></table>

## pyWinAuto範例程式

[https://github.com/pywinauto/pywinauto/tree/298bb017f91f30cc524988bfa582ee46d7aed775/examples](https://github.com/pywinauto/pywinauto/tree/298bb017f91f30cc524988bfa582ee46d7aed775/examples)


## 来源  (http://www.qa-knowhow.com/?p=2559)


 